<a href="https://colab.research.google.com/github/HYUNMIN-HWANG/Colab/blob/main/DACON_vision2/vision2_0226_2_contour_2test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('./MyDrive')

Mounted at ./MyDrive


# vision2 dirty test data 예측해보자

In [ ]:
# warnings.filterwarnings("ignore")
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
import cv2
import gc
from keras import backend as bek
from sklearn.model_selection import train_test_split, KFold
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV



In [ ]:
idg = ImageDataGenerator(height_shift_range=(-1,1), 
                        width_shift_range=(-1,1),
                        rotation_range=120,
                        fill_mode='nearest'
                        )
idg2 = ImageDataGenerator()

In [ ]:
# 잘 예측하는지 확인 
import matplotlib.pyplot as plt
import matplotlib.image as img
from glob import glob 

# submission
sub = pd.read_csv('/content/MyDrive/MyDrive/Colab/DACON_vision2/sample_submission.csv')
# print(sub.head())

model = load_model('/content/MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0226_1_vison1_0.3941.hdf5') # <<-- 여기에 데이콘2 테스트 데이터를 넣어서 숫자를 예측한다.

n = 50000
row = 0
for i in range(5000) :
    test_img=glob(f'/content/MyDrive/MyDrive/Colab/DACON_vision2/contour/{n}_*.png')
    print(len(test_img))

    predict_list = []

    for img in test_img :
        pred_img = cv2.imread(img) # 

        # print(pred_img.shape)   # (64, 64, 3)
        plt.imshow(pred_img)
        cv2.waitKey(0)

        pred_img = pred_img.astype('float32')
        pred_img = cv2.resize(pred_img, (56,56))
        # print(pred_img.shape)   # (56, 56)

        pred_img = pred_img.reshape(1, pred_img.shape[0], pred_img.shape[1], pred_img.shape[2])/255.
        # print(pred_img.shape)   # (1, 56, 56, 3)
        pred_generator = idg2.flow(pred_img, shuffle=False)

        result = model.predict_generator(pred_generator, verbose=True)
        print(result.max())
        if result.max() < 0.6 : # 최대 예측값이 60%를 넘지 않는다면 이상한 문자이므로 패쓰한다.
            pass
        else :
            result_max = result.argmax(1)[0]
            print(result_max) 
            predict_list.append(result_max)

    # 중복값 제거하기 위해서 집합으로 만들었다가 리스트로 변환
    predict_set = set(predict_list)
    predict_list = list(predict_set)
    print(predict_list)

    # submission 형태로 만들기
    result = np.zeros((1,26), dtype=np.int8)
    # print(result)

    for idx in predict_list :
        result[0][idx] = 1
    print(result)

    sub.iloc[row][1:] = result[0]
    sub.to_csv('/content/MyDrive/MyDrive/Colab/DACON_vision2/sub_0227_2.csv', index=False)
    print(sub.head())

    n += 1
    row += 1

print("==Done==")